In [1]:
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
from tqdm.contrib import tzip
import numpy as np

WD = 'C:\\Users\Sunwoo\Desktop\업무\code\[코드스테이츠] AIB16\AI_16_허선우_Section2_Project\Data'
os.chdir(WD)

In [2]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
import requests
import time
from lxml import etree
import os

In [7]:
dataset_financial = pd.read_csv('dataset_finacial.csv', index_col = 0).iloc[:,:-9]
dataset_financial['결산년도'] = pd.to_datetime(dataset_financial['결산년도'])
dataset_financial = dataset_financial.drop_duplicates()
df = dataset_financial['사업자등록번호'].drop_duplicates()
df

0     221-81-48293
5     507-87-01423
7     450-88-01196
9     495-86-02103
10    587-81-02009
          ...     
0     301-81-62403
0     124-81-72250
0     301-81-86915
0     301-81-27032
0     315-81-32820
Name: 사업자등록번호, Length: 83321, dtype: object

In [69]:
def make_dataset(wd):
    
    wd_list = sorted(glob(wd))
    df = pd.DataFrame()
    
    for fn in tqdm(wd_list):
        
        df_new = pd.read_csv(fn, index_col = 0)
        df = pd.concat([df, df_new])
        
    return df

def update_dataset(wd):
    
    bsno_dataset = pd.read_csv('bsno_dataset.csv', index_col = 0)
    bsno_dataset2 = make_dataset(wd)
    
    bsno_dataset = pd.concat([bsno_dataset, bsno_dataset2]).drop_duplicates()
    bsno_dataset.to_csv('bsno_dataset.csv')
    
    return bsno_dataset

def crawler(crawl_df, bsno_dataset):
    url =  'https://teht.hometax.go.kr/websquare/websquare.html?w2xPath=/ui/ab/a/a/UTEABAAA13.xml'
    
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(5)
    driver.get(url)

    bsno_ = bsno_dataset['사업자등록번호'].tolist()

    for bizno in tqdm(crawl_df.tolist()):

        if bizno in bsno_:
            continue

        else:
            bsno = driver.find_element(By.XPATH, '//*[@id="bsno"]')
            bsno.clear()
            bsno.send_keys(bizno)
            bsno.send_keys(Keys.RETURN)
            time.sleep(0.12)

            try : 
                html = driver.page_source
                pd.read_html(html)[0].to_csv(".\Bizcode\"+ str(bizno) +".csv")
            except: continue
                
    return bsno_dataset

In [74]:
#데이터셋 불러오기 +업데이트
bsno_dataset = update_dataset(wd = "Bizcode/*.csv")

#데이터 적재
bsno_dataset = crawler(crawl_df = df, bsno_dataset = bsno_dataset)

#데이터셋 저장
bsno_dataset.to_csv('bsno_dataset.csv')

100%|██████████| 14/14 [00:00<00:00, 518.41it/s]
C:\Users\Sunwoo\AppData\Local\Temp\ipykernel_13356\1041419733.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
100%|██████████| 83321/83321 [00:40<00:00, 2062.73it/s]


In [79]:
bsno_dataset['폐업일자'] = bsno_dataset['사업자등록상태'].str.split('폐업일자:').str[1].str.extract(r'(\d+-\d+-\d+)')
bsno_dataset['폐업일자'] = pd.to_datetime(bsno_dataset['폐업일자'])

bsno_dataset['폐업여부'] = (~(bsno_dataset['폐업일자'] <= '2019-12-31')).astype(int)
bsno_dataset.set_index('사업자등록번호').to_csv('biz_no_2019.csv')

bsno_dataset['폐업여부'] = (~(bsno_dataset['폐업일자'] <= '2020-12-31')).astype(int)
bsno_dataset.set_index('사업자등록번호').to_csv('biz_no_2020.csv')

bsno_dataset['폐업여부'] = (~(bsno_dataset['폐업일자'] <= '2021-12-31')).astype(int)
bsno_dataset.set_index('사업자등록번호').to_csv('biz_no_2021.csv')

bsno_dataset['폐업여부'] = (~(bsno_dataset['폐업일자'] <= '2022-12-31')).astype(int)
bsno_dataset.set_index('사업자등록번호').to_csv('biz_no_2022.csv')

In [83]:
bsno_dataset

,사업자등록번호,사업자등록상태,조회기준일자,폐업일자,폐업여부
0,101-81-00452,부가가치세 일반과세자 입니다.,2022-12-04,NaT,1
0,101-81-24696,부가가치세 일반과세자 입니다.,2022-12-04,NaT,1
0,101-81-62477,부가가치세 일반과세자 입니다.,2022-12-04,NaT,1
0,105-81-68075,부가가치세 일반과세자 입니다.,2022-12-04,NaT,1
0,128-86-79624,부가가치세 일반과세자 입니다.,2022-12-04,NaT,1
...,...,...,...,...,...
0,767-81-01284,부가가치세 일반과세자 입니다.,2022-12-07,NaT,1
0,873-86-01911,부가가치세 일반과세자 입니다.,2022-12-07,NaT,1
0,108-81-83610,부가가치세 일반과세자 입니다.,2022-12-07,NaT,1
0,120-81-01336,부가가치세 일반과세자 입니다.,2022-12-07,NaT,1
